In [21]:
import flymovie as fm
import numpy as np
import scipy.ndimage as ndi
%load_ext autoreload
%autoreload 2

def calc_nM(stack, mask, im_dims, brightness, kernel_1gfp):
    """Calculate concentration in nM of labeled species."""
    pix = stack[mask > 0]
    num_pix = len(pix)
    sum_ = np.sum(pix)
    rel_brightness = brightness / 33.54
    nmolecules = sum_ / np.sum(kernel_1gfp) / rel_brightness

    # multiply each side length, 1e-27 is 1e-9 meters cubed to convert nm to m, 1000 is liters in a cubic meter
    L = np.product(im_dims) * 1e-27 * 1000 * num_pix # in liters
    mols = nmolecules / 6.022e23
    M = mols / L
    nM = M / 1e-9
    return nM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Add files and params

In [9]:
kernel_peak = 28_000
n_gfps = 60
# 34 for egfp, 93 for mneon, 54 for sfgfp
brightness = 93
kernelfile = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220210_21x25x25pixels_100x50x50voxel.pkl'
stackfile = '/Volumes/stad3/2022-03-21/26983-1-1-em2-16.czi'
kernel_dims = (100, 50, 50)
im_dims = (250, 85, 85)

### Calculate kernel for 1 GFP.

In [8]:
kernel = fm.load_pickle(kernelfile)
kernel = ndi.zoom(kernel, np.divide(kernel_dims, im_dims), order=3)
kernel_1gfp = kernel * (kernel_peak / np.max(kernel)) / n_gfps

### Smooth and view stack, trimming in Z if necessary/desired

In [11]:
stack = fm.read_czi(stackfile, swapaxes=False)
stack = stack[0:]
sm = ndi.gaussian_filter(stack, (2,5,5))
fm.viewer(sm, 5)

interactive(children=(Dropdown(description='Color', index=8, options=('Greens', 'gray_r', 'plasma', 'viridis',…

### Set threshold and make mask.

In [15]:
t = 3400
mask = np.where(sm > t, 1, 0)
fm.viewer([np.where(mask, stack, 0), stack], 10)

interactive(children=(Dropdown(description='Color', index=8, options=('Greens', 'gray_r', 'plasma', 'viridis',…

### Calculate concentration

In [22]:
calc_nM(stack, mask, im_dims, brightness, kernel_1gfp)

60.30028913112654

### bicoid (25k - 60)
/Volumes/stad3/2022-03-18/bcd-gfp-em2-11.czi 155 nM (25k - 60)

### rpb1 (40k - 120)
/Volumes/stad3/2022-03-18/rpb1-gfp-em1-21.czi 385 nM (nc14)

/Volumes/stad3/2022-03-18/rpb1-gfp-em2-02.czi 335 nM (early, maybe nc11)

/Volumes/stad3/2022-03-18/rpb1-gfp-em2-06.czi (early)

### zld (25k - 60)
/Volumes/stad3/2022-03-18/zld-mneon-em1-01.czi 83 nM

/Volumes/stad3/2022-03-18/zld-mneon-em1-10.czi 79 nM

/Volumes/stad3/2022-03-18/zld-mneon-em1-14.czi 75 nM

### h2b (4k - 120)
/Volumes/stad3/2022-03-17/h2b-gfp-em1-11.czi 5000 nM

/Volumes/stad3/2022-03-17/h2b-gfp-em2-08.czi 4977 nM

### cp190 (20k - 60)

/Volumes/stad3/2022-03-24/cp190-em3-01.czi 215 nM

/Volumes/stad3/2022-03-24/cp190-em3-08.czi 218 nM

/Volumes/stad3/2022-03-24/cp190-em3-16.czi 212 nM

/Volumes/stad3/2022-03-24/cp190-em3-24.czi 210 nM

### zld cterm (28k - 60)

26983-1-1-em2-01.czi 86 nM

26983-1-1-em2-01.czi 75.8 nM

26983-1-1-em2-16.czi 60 nM


